In [ ]:
import numpy as np
import cv2
import h5py
import keras
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
# Read back in data set
f = h5py.File('sim_tl_data.hdf5', 'r')

train_images = f["images"]
labels = f["labels"]

In [ ]:
# Change to numpy arrays so keras can use
train_images = np.array(train_images)
labels = np.array(labels)

In [ ]:
# Checking distribution
import matplotlib.pyplot as plt
%matplotlib inline

plt.hist(labels, bins = 4)

In [ ]:
# IF GRAYSCALE ONLY
#train_images = train_images[..., np.newaxis]

In [ ]:
# One-hot encode labels
labels = keras.utils.to_categorical(labels)

In [ ]:
# Shuffle images along with their labels, then split into training/validation sets
train_images, labels = shuffle(train_images, labels)

X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.2)

In [ ]:
# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv2D, Flatten, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

# Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
batch_size = 30
epochs = 25
pool_size = (2, 2)
input_shape = X_train.shape[1:]

In [ ]:
# Here is the actual neural network
model = Sequential()
# Normalizes incoming inputs. First layer needs the input shape to work
model.add(BatchNormalization(input_shape=input_shape))

# Conv Layer 1
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding="valid"))
model.add(Activation('relu'))
#model.add(Dropout(0.2))

# Conv Layer 2
model.add(Conv2D(32, (3, 3), strides=(1, 1), padding="valid"))
model.add(Activation('relu'))

# Pooling
model.add(MaxPooling2D(pool_size=pool_size))

# Conv Layer 3
model.add(Conv2D(16, (3, 3), strides=(1, 1), padding="valid"))
model.add(Activation('relu'))

# Conv Layer 4
model.add(Conv2D(8, (3, 3), strides=(1, 1), padding="valid"))
model.add(Activation('relu'))

# Pooling
model.add(MaxPooling2D(pool_size=pool_size))

# Flatten and Dropout
model.add(Flatten())
model.add(Dropout(0.3))

# FC Layer 1
model.add(Dense(64))
model.add(Activation('relu'))
#model.add(Dropout(0.3))

# FC Layer 2
model.add(Dense(32))
model.add(Activation('relu'))

# Final FC Layer - five classes
model.add(Dense(4))
model.add(Activation('softmax'))

# Using a generator to help the model generalize/train better
datagen = ImageDataGenerator(rotation_range = 10, vertical_flip = True, channel_shift_range = 0.1)
datagen.fit(X_train)

# Compiling and training the model
model.compile(optimizer='Adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), samples_per_epoch = len(X_train), nb_epoch=epochs, verbose=1, validation_data=(X_val, y_val))

In [ ]:
model.summary()

In [ ]:
# Save model architecture and weights
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')